<a href="https://colab.research.google.com/github/naina-dutraj/data-analysis_Beautiful_soup/blob/main/beautifulS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup


In [2]:
import requests
import pandas as pd


In [3]:
def df_extractor(page_num):
  response = requests.get(f'https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors/page/{num}')
  web_page = response.text
  soup = BeautifulSoup(web_page,'html.parser')
  table = soup.find_all('table')[0]
  dataframe = pd.read_html(str(table))[0]
  return dataframe



In [4]:
def clean_df(df):
  df.rename(columns={df.columns[3] : 'Early Career Pay', df.columns[4] :'Mid-Career Pay', df.columns[5] : '% High Meaning'},inplace = True)
  for index in range(len(df.columns)):
    name = df.columns[index]
    df[name] = df[name].str.replace(f'{name}:','')
    try:
      df[name] = df[name].str.replace('$','')
      df[name] = df[name].str.replace('%','')
      df[name] = df[name].str.replace(',','_')
    except:
      continue
  return df


In [5]:
response = requests.get('https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors')
web_page = response.text
soup = BeautifulSoup(web_page,'html.parser')
table = soup.find_all('table')[0]
df1 = pd.read_html(str(table))[0]
df1 = clean_df(df1)


<ipython-input-4-258bcb088b79>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[name] = df[name].str.replace('$','')


In [6]:
for num in range(2,35):
  df2 = clean_df(df_extractor(num))
  df = pd.concat([df1,df2])
  df1=df

combined_df = df1

<ipython-input-4-258bcb088b79>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[name] = df[name].str.replace('$','')
<ipython-input-4-258bcb088b79>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[name] = df[name].str.replace('$','')
<ipython-input-4-258bcb088b79>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[name] = df[name].str.replace('$','')
<ipython-input-4-258bcb088b79>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular 

In [7]:
combined_df.shape

(827, 6)

In [8]:
combined_df = combined_df.set_index(pd.Index(range(len(combined_df))))
combined_df

,Rank,Major,Degree Type,Early Career Pay,Mid-Career Pay,% High Meaning
0,1,Petroleum Engineering,Bachelors,93_200,187_300,67
1,2,Operations Research & Industrial Engineering,Bachelors,84_800,170_400,28
2,3,Electrical Engineering & Computer Science (EECS),Bachelors,108_500,159_300,46
3,4,Interaction Design,Bachelors,68_300,155_800,55
4,5,Public Accounting,Bachelors,59_800,147_700,47
...,...,...,...,...,...,...
822,823,Outdoor Education,Bachelors,37_400,46_300,52
823,824,Early Childhood Education,Bachelors,36_100,45_400,78
824,825,Mental Health,Bachelors,36_900,45_000,-
825,826,Medical Assisting,Bachelors,36_000,44_800,-


In [9]:
combined_df.head()

,Rank,Major,Degree Type,Early Career Pay,Mid-Career Pay,% High Meaning
0,1,Petroleum Engineering,Bachelors,93_200,187_300,67
1,2,Operations Research & Industrial Engineering,Bachelors,84_800,170_400,28
2,3,Electrical Engineering & Computer Science (EECS),Bachelors,108_500,159_300,46
3,4,Interaction Design,Bachelors,68_300,155_800,55
4,5,Public Accounting,Bachelors,59_800,147_700,47


In [10]:
import numpy as np
combined_df = combined_df.replace('-', np.nan)
combined_df.to_csv('my_dataframe.csv', index=False, na_rep='NULL')

In [11]:
combined_df.isna().sum()

Rank                 0
Major                0
Degree Type          0
Early Career Pay     0
Mid-Career Pay       0
% High Meaning      64
dtype: int64

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0)

for name in combined_df.columns:
  if combined_df[name].isna().sum() > 0:
    combined_df[name] = imputer.fit_transform(combined_df[[name]])

In [13]:
combined_df.isna().sum()

Rank                0
Major               0
Degree Type         0
Early Career Pay    0
Mid-Career Pay      0
% High Meaning      0
dtype: int64

In [14]:
combined_df[combined_df.columns[0]] = combined_df[combined_df.columns[0]].astype(int)
combined_df[combined_df.columns[3]] = combined_df[combined_df.columns[3]].astype(int)
combined_df[combined_df.columns[4]] = combined_df[combined_df.columns[4]].astype(int)
combined_df[combined_df.columns[5]] = combined_df[combined_df.columns[5]].astype(int)

In [15]:
combined_df['Early Career Pay'].idxmax()

2

In [16]:
combined_df.loc[2]

Rank                                                               3
Major               Electrical Engineering & Computer Science (EECS)
Degree Type                                                Bachelors
Early Career Pay                                              108500
Mid-Career Pay                                                159300
% High Meaning                                                    46
Name: 2, dtype: object

In [17]:
combined_df['Early Career Pay'].idxmin()

804

In [18]:
combined_df.loc[804]

Rank                          805
Major               Voice & Opera
Degree Type             Bachelors
Early Career Pay            34500
Mid-Career Pay              53300
% High Meaning                 57
Name: 804, dtype: object

In [19]:
combined_df['Mid-Career Pay'].idxmax()

0

In [20]:
combined_df.loc[0]

Rank                                    1
Major               Petroleum Engineering
Degree Type                     Bachelors
Early Career Pay                    93200
Mid-Career Pay                     187300
% High Meaning                         67
Name: 0, dtype: object

In [21]:
combined_df['Mid-Career Pay'].idxmin()

826

In [22]:
combined_df.loc[826]

Rank                          827
Major               Metalsmithing
Degree Type             Bachelors
Early Career Pay            40000
Mid-Career Pay              40300
% High Meaning                 33
Name: 826, dtype: object